# Gauss-Levenberg-Marquardt (GLM) algorithm

The Gauss-Levenberg-Marquart (GLM) method is a derivative-based method used to search the objective function surface for its minimum value. It assumes that simulated values of observation targets vary continuously in response to changes in calibration-adjusted model parameters. 

However, certain combinations of parameters can provide similar fits to calibration data (as we saw in the "freyberg k and r" tutorial, in which we attempt to calibrate a model with two correlated parameters). This phenomenon is refered to as non-uniqueness. For a two-parameter case, this can be visualized as a "trough" in the objective function surface. In such cases, there is no single "minimum" value for the objective function. #dispair

Nevertheless, PEST(++) may obtain a solution to an ill-posed inverse problem and even fit the calibration dataset very well. This occurs due to the addition of the Marquardt lambda to the diagonal of the $J^{t}QJ$ matrix and enables the achievement of a set of parameters which minimize the objective function. However, this parameter set will not be unique. Changing observation weights, initial parameter values, or a different Marquardt lambda selection strategy, may result in an entirely different set of parameters which also minimize the objective function.

In practice, this can be avoided by ensuring a well-posed inverse problem. In simple terms, this requires having a greater number of sources of calibration data than the number of adjustable parameters. As we saw in the "freyberg k, r and flux observations" tutorial, one option is to include multiple types of observation data which inform different parameter types. (As we will see further on, regularisation provides another.)

In this tutorial we are going to visualize the "objective function surface" and how the parameter ugrade process moves across it. 

### Admin
We have provided some pre-cooked PEST dataset files, wrapped around the modified Freyberg model. This is the same dataset introduced in the "freyberg_pest_setup" and "freyberg_k" notebooks. 

The functions in the next cell import required dependencies and prepare a folder for you. This folder contains the model files and a preliminary PEST setup. Run the cells, then inspect the new folder named "freyberg_mf6" which has been created in your tutorial directory. (Just press `shift+enter` to run the cells). 

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import shutil

# sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd
import response_surface as resurf

plt.rcParams['font.size'] = 10
pyemu.plot_utils.font =10

In [ ]:
# folder containing original model files
org_d = os.path.join('..', '..', 'models', 'monthly_model_files_1lyr_newstress')
# a dir to hold a copy of the org model files
tmp_d = os.path.join('freyberg_mf6')


run_response_surfaces = False

if run_response_surfaces==False:
    print('Assuming PEST++SWP has bene run already and the folder with files is available')
else:
    if os.path.exists(tmp_d):
        shutil.rmtree(tmp_d)
    shutil.copytree(org_d,tmp_d)
    # get executables
    hbd.prep_bins(tmp_d)
    # get dependency folders
    hbd.prep_deps(tmp_d)
    # run our convenience functions to prepare the PEST and model folder
    hbd.prep_pest(tmp_d)

### Reminder - the modified-Freyberg model
Just a quick reminder of what the model looks like and what we are doing. 

It is a one-layer model. A river runs north-south, represented with the SFR package (green cells in the figure). On the southern border there is a GHB (cyan cells). No-flow cells are shown in black. Pumping wells are shown with red cells. 

Time-series of measured heads are available at the locations marked with black X's. River flux is also measured at three locations (headwater, tailwater and gage; not displayed).

The simulation starts with a steady state stress period, followed by twelve transient stress periods. These represent the historic period, for which measured data are available.

A subsequent twelve transient stress periods representing a period in the future. Modelling is undertaken to assess selected forecasts during the simulated period.

In [ ]:
hbd.plot_freyberg(tmp_d)

### The PEST Control File

You may  wish to explore the `freyberg_mf6` folder which has been created in the tutorial directory. In it you will find a PEST control file named `freyberg.pst`.

Let's use `pyemu` to load the PEST control file and check some details. 

In [ ]:
pst = pyemu.Pst(os.path.join(tmp_d, 'freyberg.pst'))
pst.par_names

In [ ]:
par = pst.parameter_data
par

We shall explore the effect of having two adjustable parameters: `hk1` and `rch0`. As we saw previously, these two parameters are correlated. If we use only head observations for calibration we are unable to achive a unique solution. 

(We shall change `hk1` parameter bounds just to make visualization easier.)

In [ ]:
par.loc['rch0', 'partrans'] = 'log'
par.loc['hk1', 'parlbnd'] = 1.5
par.loc['hk1', 'parubnd'] = 15

Re-write the control file:

In [ ]:
pst.write(os.path.join(tmp_d, 'freyberg.pst'),version=2)

The `response_surface.py` file in the tutorial folder contains a few functions to run PEST++SWP and make plots. These run PEST++SWP a few hundred times for a combination of `hk1` and `rch0` values.

As with the other notebooks that run modflow in parallel, please change the number of workers to be in harmony with your computer!

In [ ]:
num_workers=20

In [ ]:
gw_respsurf_d = "gw_only_respsurf"
if run_response_surfaces==False:
    print('Assuming PEST++SWP has bene run already and the folder with files is available')
else:
    if os.path.exists(gw_respsurf_d):
        shutil.rmtree(gw_respsurf_d)
    shutil.copytree(tmp_d,gw_respsurf_d)
    resurf.run_respsurf(par_names=['hk1','rch0'],WORKING_DIR=gw_respsurf_d,num_workers=num_workers)

Make a plot of the response surface for `hk1` (x-axis) and `rch0` (y-axis). The colored contours indicate the objective function value for each combination of these two parameters. 

As you can see, a long eliptical "trough of dispair" of optimal values is formed. Parameter combinations in this zone all result in equivalent levels of "good fit" to the observation dataset.  The trough of dispair is an example of non-uniqueness in graphically form.  A problem that arises is while many combinations of recharge and HK can fit the observation dataset, the forecasts of interest model with "calibrated" model could be highly sensitive to the value of HK and/or recharge and single "calibrated" model cant represent this non-uniqueness under forecasting conditions.  #uncertaintyanalysis

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=gw_respsurf_d,
                                                  title="Only groundwater level observations in $\\Phi$") #maxresp=1e3,

### Add Flux Observations

As we did in the "freyberg k, r and flux obs" tutorial, we know add a second set of observation data. These are measurments of stream flow. We now have observations of head and observations of stream flow

In [ ]:
pst = pyemu.Pst(os.path.join(tmp_d, 'freyberg.pst'))

In [ ]:
# set weights to gage-1 observations during calibration period
obs = pst.observation_data
obs_times = obs.loc[~obs['gage-1'].isnull(), 'gage-1'].astype(float).values
calib_times = [str(i) for i in obs_times if i<4018.5]
obs.loc[obs['gage-1'].isin(calib_times), 'weight'] = 0.003

Re-write the control file.

In [ ]:
pst.write(os.path.join(tmp_d, 'freyberg.pst'),version=2)

Run PEST++SWP again to recalculate the response surface.

In [ ]:
bal_respsurf_d = "bal_respsurf"
if run_response_surfaces==False:
    print('Assuming PEST++SWP has been run already and the folder with files is available')
else:
    if os.path.exists(bal_respsurf_d):
        shutil.rmtree(bal_respsurf_d)
    shutil.copytree(tmp_d,bal_respsurf_d)
    resurf.run_respsurf(par_names=['hk1','rch0'],WORKING_DIR=bal_respsurf_d,num_workers=num_workers)

And plot it up again. Now we see the objective function surface funneling down to a single point. We have achieved a unique solution.  The "trough of dispair" has been the "bowl of uniqueness"!  A clear demonstration of the value of unique and diverse data...

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d,
                                                 title="Groundwater level and streamflow observations in $\\Phi$") #maxresp=1e3,

# How weights change the response surface

We can think of adding observations as the extreme case of changing weights from zero to not zero. So lets do one last last response surface run, where we skew the objective function to mostly composed of surface water flow observations - we will do this by increasing the weight on those observations:

In [ ]:
# obs = pst.observation_data
# for grp in pst.nnz_obs_groups:
#     if "gage" not in grp:
#         continue
#     oobs = obs.loc[obs.obgnme==grp,:]
#     oobs = oobs.loc[oobs.weight>0,:]
#     mn = oobs.obsval.mean()
#     lower_names = oobs.loc[oobs.obsval<mn,"obsnme"]
#     assert len(lower_names) > 0
#     obs.loc[lower_names,"weight"] = 100
obs = pst.observation_data
obs_times = obs.loc[~obs['gage-1'].isnull(), 'gage-1'].astype(float).values
calib_times = [str(i) for i in obs_times if i<4018.5]
ovals = obs.loc[obs['gage-1'].isin(calib_times), 'obsval']
obs.loc[obs['gage-1'].isin(calib_times), 'weight'] = 1./(ovals*0.01)
pst.write(os.path.join(tmp_d, 'freyberg.pst'),version=2)
imbal_respsurf_d = "imbal_respsurf"

if run_response_surfaces==False:
    print('Assuming PEST++SWP has been run already and the folder with files is available')
else:
    if os.path.exists(imbal_respsurf_d):
        shutil.rmtree(imbal_respsurf_d)
    shutil.copytree(tmp_d,imbal_respsurf_d)
    resurf.run_respsurf(par_names=['hk1','rch0'],WORKING_DIR=imbal_respsurf_d,num_workers=num_workers)

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=imbal_respsurf_d,
                                                 title="Streamflow-focused $\\Phi$") #maxresp=1e3,

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(12,4))
_ = resurf.plot_response_surface(WORKING_DIR=gw_respsurf_d,
                                                  title="Only groundwater level observations\n in $\\Phi$",ax=axes[0]) #maxresp=1e3,

_ = resurf.plot_response_surface(WORKING_DIR=bal_respsurf_d,ax=axes[1],
                                                 title="Groundwater level and streamflow\n observations in $\\Phi$") #maxresp=1e3,
_ = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=imbal_respsurf_d,ax=axes[2],
                                                 title="Streamflow-focused $\\Phi$") #maxresp=1e3,
plt.tight_layout()
plt.savefig("respsurf_combined.pdf")
plt.show()
plt.close(fig)

# Understanding Lambdas

When used to undertake highly parameterized inversion, PESTPP-GLM implements theory and methodologies that are programmed into PEST. Some theory, employing matrices and vectors, is used to describe the linearized inverse problem on which so-called “gradient methods” are based. Through repeated linearization of the inverse problem over successive iterations, these methods achieve their purpose of model calibration, notwithstanding the nonlinear relationship that exists between model outputs and model parameters.  It should also be noted that PESTPP-IES also implements the GLM solution, but an ensemble of parameter vectors.  So the single trajectory below can be thought of as one of the many tradjectories that ensemble of parameter vectors take.

Nonlinear model behaviour is also accommodated by introducing a so-called "Marquardt lambda" to these equations. Employing a nonzero lambda tweaks the direction of parameter improvement so that it is more aligned with the objective function gradient. This increases the efficiency of early iterations of the inversion process when implemented in conjunction with a nonlinear model.

If Lambdas are high, then the parameter estimation process updates parameter values accodring to the objective function gradient. If they are small (or zero), parameter upgrades receive a greater influence from the $J^{t}QJ$ matrix. 

Highy non-linear problems, in particular, benefit from high lambdas at the start of the parameter estimation process. This accomodates non-elipticity of the objective function surface, avoiding sending the parameter estimation process in the "wrong direction" and wasting computation time. 

However, as the objective function minimum is approached, the process becomes more eficient if smaller lambdas are used. This avoids the phenomenon known as "hemstitching", in which parameter upgrades jump-across small, thin "troughs" in the objective function surface. 

Note again, the effect of lambda on the parameter upgrade is the same in PESTPP-IES.

See the PEST Book (Doherty, 2015) and the PEST++ users manual for more details.

### No Lambda

Let's visualize the effect of lambda values on the upgrade process. First, with no lambda added to the parameter upgrade equations.

In [ ]:
# turn off the Maquardt lambda
pst.pestpp_options['lambdas']=0.0
pst.pestpp_options['lambda_scale_fac']=1.0
pst.pestpp_options['upgrade_augment']='False'
# set weights to gage-1 observations during calibration period
obs = pst.observation_data
obs_times = obs.loc[~obs['gage-1'].isnull(), 'gage-1'].astype(float).values
calib_times = [str(i) for i in obs_times if i<4018.5]
obs.loc[obs['gage-1'].isin(calib_times), 'weight'] = 0.003
par = pst.parameter_data
pst.control_data.facparmax = 10000
pst.control_data.relparmax = 10000

Initial parameter values specify where the parameter estimation process "starts":

In [ ]:
# to make a more compeling plotc
pst.parameter_data.loc['hk1', 'parval1'] = 14.5
pst.parameter_data.loc['rch0', 'parval1'] = 1.99

Write and run:

In [ ]:
pst.control_data.noptmax=250
pst.write(os.path.join(tmp_d, 'freyberg.nolam.pst'),version=2)
working_d = "glm_nolam"

if os.path.exists(working_d):
    shutil.rmtree(working_d)
shutil.copytree(tmp_d,working_d)
pyemu.os_utils.run('pestpp-glm freyberg.nolam.pst', cwd=working_d)

Plot the response surface:

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d) #maxresp=1e3,
pars, obfun = resurf.add_trajectory_to_plot(fig, ax, 'Trajectory with no $\\lambda$', pst_name='freyberg.nolam.pst',working_dir=working_d)

In [ ]:
df_obj = pd.read_csv(os.path.join(working_d, "freyberg.nolam.iobj"), index_col=0)
df_obj.loc[:,["total_phi","model_runs_completed"]].plot(subplots=True, marker='x')

### Moderate Lambda

In [ ]:
# set a moderate lambda value
pst.pestpp_options['lambdas']=700.0
pst.pestpp_options['lambda_scale_fac']=1.0
pst.pestpp_options['upgrade_augment']='False'

pst.write(os.path.join(tmp_d, 'freyberg.modlam.pst'),version=2)
working_d = "glm_modlam"


if os.path.exists(working_d):
    shutil.rmtree(working_d)
shutil.copytree(tmp_d,working_d)
pyemu.os_utils.run('pestpp-glm freyberg.modlam.pst', cwd=working_d)

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d) #maxresp=1e3,
pars, obfun = resurf.add_trajectory_to_plot(fig, ax, 'Trajectory with moderate $\\lambda$', pst_name='freyberg.modlam.pst',working_dir=working_d)

In [ ]:
df_obj = pd.read_csv(os.path.join(working_d, "freyberg.modlam.iobj"), index_col=0)
df_obj.loc[:,["total_phi","model_runs_completed"]].plot(subplots=True, marker='x')

### Large Lambda

In [ ]:
# set a large lambda value
pst.pestpp_options['lambdas']=7000.0
pst.pestpp_options['lambda_scale_fac']=1.0
pst.pestpp_options['upgrade_augment']='False'

pst.write(os.path.join(tmp_d, 'freyberg.largelam.pst'),version=2)
working_d = "glm_largelam"


if os.path.exists(working_d):
    shutil.rmtree(working_d)
shutil.copytree(tmp_d,working_d)
pyemu.os_utils.run('pestpp-glm freyberg.largelam.pst', cwd=working_d)

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d) #maxresp=1e3,
pars, obfun = resurf.add_trajectory_to_plot(fig, ax, 'Trajectory with large $\\lambda$', pst_name='freyberg.largelam.pst',working_dir=working_d)

In [ ]:
df_obj = pd.read_csv(os.path.join(working_d, "freyberg.largelam.iobj"), index_col=0)
df_obj.loc[:,["total_phi","model_runs_completed"]].plot(subplots=True, marker='x')

### Normal PEST(++) Behaviour

In practice, PEST and PEST++ can search for the optimal parameter upgrade vector at each iteration. PEST(++) calculates upgrade vectors, each based on a different value of lambda. The one which results in the largest objective function decrease is used to calculate parameters for the next iteration. 

In [ ]:
pst.pestpp_options['lambdas']= [0.1, 1, 10,100]
pst.pestpp_options['lambda_scale_fac']=[0.9, 0.8, 0.7, 0.5]
pst.pestpp_options['upgrade_augment']='True'

pst.write(os.path.join(tmp_d, 'freyberg.testlam.pst'),version=2)
working_d = "glm_testlam"

if os.path.exists(working_d):
    shutil.rmtree(working_d)
shutil.copytree(tmp_d,working_d)
pyemu.os_utils.run('pestpp-glm freyberg.testlam.pst', cwd=working_d)

In [ ]:
fig, ax, resp_surf = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d) #maxresp=1e3,
pars, obfun = resurf.add_trajectory_to_plot(fig, ax, 'Trajectory with multiple $\\lambda$ values tested and automatic $\\lambda$ tuning', pst_name='freyberg.testlam.pst',working_dir=working_d)

In [ ]:
df_obj = pd.read_csv(os.path.join(working_d, "freyberg.testlam.iobj"), index_col=0)
df_obj.loc[:,["total_phi","model_runs_completed"]].plot(subplots=True, marker='x')

In [ ]:
fig,axes = plt.subplots(2,2,figsize=(8.5,8.5))
axes = axes.flatten()
_ = resurf.plot_response_surface(WORKING_DIR=bal_respsurf_d,ax=axes[0]) 

df_obj_nolam = pd.read_csv(os.path.join("glm_nolam", "freyberg.nolam.iobj"))
pars, obfun = resurf.add_trajectory_to_plot(fig, axes[0], 'A) Trajectory with no $\\lambda$\n({0} iterations)'.\
                                            format(df_obj_nolam.iteration.max()), pst_name='freyberg.nolam.pst',working_dir="glm_nolam")

_ = resurf.plot_response_surface(WORKING_DIR=bal_respsurf_d,ax=axes[1])
df_obj_modlam = pd.read_csv(os.path.join("glm_modlam", "freyberg.modlam.iobj"))
pars, obfun = resurf.add_trajectory_to_plot(fig, axes[1], 'B) Trajectory with moderate $\\lambda$\n({0} iterations)'.\
                                            format(df_obj_modlam.iteration.max()), pst_name='freyberg.modlam.pst',working_dir="glm_modlam")

_ = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d,ax=axes[2])
df_obj_largelam = pd.read_csv(os.path.join("glm_largelam", "freyberg.largelam.iobj"))
pars, obfun = resurf.add_trajectory_to_plot(fig, axes[2], 'C) Trajectory with large $\\lambda$\n({0} iterations)'.\
                                            format(df_obj_largelam.iteration.max()), pst_name='freyberg.largelam.pst',working_dir="glm_largelam")

_ = resurf.plot_response_surface(figsize=(7,7),WORKING_DIR=bal_respsurf_d,ax=axes[3])
df_obj_testlam = pd.read_csv(os.path.join("glm_testlam", "freyberg.testlam.iobj"))
pars, obfun = resurf.add_trajectory_to_plot(fig, axes[3], 'D) Trajectory with $\\lambda$ testing and tuning\n({0} iterations)'.\
                                            format(df_obj_testlam.iteration.max()), pst_name='freyberg.testlam.pst',working_dir="glm_testlam")


plt.tight_layout()
plt.savefig("respsurf_traj_combined.pdf")
plt.show()
plt.close(fig)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8.5,4))
labels = ["Trajectory with no $\\lambda$",'Trajectory with moderate $\\lambda$',"Trajectory with large $\\lambda$", 'Trajectory with $\\lambda$ testing and tuning']
for df,label in zip([df_obj_nolam,df_obj_modlam,df_obj_largelam,df_obj_testlam],labels):
    ax.plot(df.iteration+1,df.total_phi,label=label,marker=".")
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_xlabel("iteration")
ax.set_ylabel("$\\Phi$")
ax.legend(loc="upper right")
plt.tight_layout()
plt.savefig("phi_history.pdf")
plt.show()
plt.close(fig)